In [1]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import imp

def change_data(data, Flag_y = True ):
    
    if (Flag_y ==True):
    
        categorical = ['top_left_square', 'top_middle_square', 'top_right_square','middle_left_square',\
             'middle_middle_square', 'middle_right_square','bottom_left_square', 'bottom_middle_square', 'bottom_right_square']
        X = data[categorical]
        X_new = pd.get_dummies(X)
        lb_make = LabelEncoder()
        data.drop(categorical,inplace=True,axis=1)
        data["class"] = lb_make.fit_transform(data["class"])
        data.reset_index(drop = True, inplace= True)
        X_new.reset_index(drop = True, inplace= True)
        bigdata= bigdata = pd.concat([X_new, data], axis= 1)
        return bigdata
    else:
        categorical = ['top_left_square', 'top_middle_square', 'top_right_square','middle_left_square',\
             'middle_middle_square', 'middle_right_square','bottom_left_square', 'bottom_middle_square', 'bottom_right_square']
        X = data[categorical]
        data.drop(categorical,inplace=True,axis=1)
        X_new = pd.get_dummies(X)
        return X_new


In [8]:
! pip install joblib

     |████████████████████████████████| 296kB 1.4MB/s eta 0:00:01


In [43]:
import os 
import pandas as pd 
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from joblib import dump,load

from sklearn import tree

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



def save_models(model_name, file):
   dump(model_name,"./{}".format(file))


def knn_model(feature, target):
    #training with train test score 
    knn = KNeighborsClassifier()
    ##grid search cross -fold
    knn_params = {"n_neighbors":np.arange(1,25)}
    knn_gscv = GridSearchCV(knn, knn_params,cv=10)
    knn_gscv.fit(feature,target.values.ravel())
    print(knn_gscv.best_params_)
    print(knn_gscv.best_score_)

    save_models(knn_gscv,"knn_model.joblib")

def random_forest(feature, target):
    clf = RandomForestClassifier(random_state = 42 )
    #clf = clf.fit(feature,target)
    parameter_grid = {'max_depth': [1, 2, 3, 4, 5, 6,7,8 ,9 ,10],
                  'max_features': [1, 2, 3, 4]}

    clf = GridSearchCV(clf,parameter_grid,cv=10)
    clf =clf.fit(feature,target.values.ravel())
    print(clf.best_params_)
    print(clf.best_score_)

    save_models(clf,"random_forest.joblib")

def predict_using_model(x,y,filename= "./knn_model.joblib"):

    
    model = load(filename)
    y_pred = model.predict(x)
    
    matrix = confusion_matrix(y , y_pred)
    print({
        "accuracy": float((matrix[0][0]+matrix[1][1])/(sum(matrix[0])+sum(matrix[1]))),
        "no_correct" : int(matrix[0][0]),
        "no_incorrect" : int(matrix[0][1]),
        "yes_correct" : int(matrix[1][1]),
        "yes_incorrect" : int(matrix[1][0])
        })

def probability_predict(x,filename = './knn_model.joblib'):
    #fi = os.getcwd()+'/model/'+filename
    fi = filename
    #change null to b 
    x.replace("null","b",inplace=True)
    x = x.applymap(lambda s:s.lower() if type(s) == str else s)
    #change missing columns with 0 
    cls_group = ['top_left_square_b', 'top_left_square_o', 'top_left_square_x',
       'top_middle_square_b', 'top_middle_square_o', 'top_middle_square_x',
       'top_right_square_b', 'top_right_square_o', 'top_right_square_x',
       'middle_left_square_b', 'middle_left_square_o', 'middle_left_square_x',
       'middle_middle_square_b', 'middle_middle_square_o',
       'middle_middle_square_x', 'middle_right_square_b',
       'middle_right_square_o', 'middle_right_square_x',
       'bottom_left_square_b', 'bottom_left_square_o', 'bottom_left_square_x',
       'bottom_middle_square_b', 'bottom_middle_square_o',
       'bottom_middle_square_x', 'bottom_right_square_b',
       'bottom_right_square_o', 'bottom_right_square_x']
    
    data =  change_data(x,False)

    new_data = pd.DataFrame()
    
    for i in cls_group:
        if i not in data.columns:
            new_data[i]=0
        else:
            new_data[i] = data[i]
    
    model = load(fi)
    new_data = np.nan_to_num(new_data)
    print(new_data)
    probab = model.predict_proba(new_data)

    return ({
        "x": probab[0][0],
        "o": probab[0][1],
    })


In [15]:
if __name__ == "__main__":
    data = pd.read_csv("./../data/model.csv")
    X, y = np.split(data, [-1], axis=1)
    seed= 42
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state = seed)   
    #random forest 
    random_forest(X_train,y_train) 
    predict_using_model(X_test,y_test)
    knn_model(X_train,y_train)
    predict_using_model(X_test,y_test,"./knn_model.joblib")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': 9, 'max_features': 4}
0.917910447761194
{'accuracy': 0.9166666666666666, 'no_correct': 81, 'no_incorrect': 14, 'yes_correct': 183, 'yes_incorrect': 10}
{'n_neighbors': 12}
0.9731343283582089
{'accuracy': 0.9861111111111112, 'no_correct': 91, 'no_incorrect': 4, 'yes_correct': 193, 'yes_incorrect': 0}


In [44]:
a=[['null','null','X','null','O','null','null','null','null']]
c = ['top_left_square', 'top_middle_square', 'top_right_square','middle_left_square','middle_middle_square', 'middle_right_square','bottom_left_square', 'bottom_middle_square', 'bottom_right_square']

df = pd.DataFrame(a,columns=c)
probability_predict(df)

[[1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0]]


{'x': 0.3333333333333333, 'o': 0.6666666666666666}